In [ ]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
np.set_printoptions(precision=2)  # reduced display precision on numpy arrays

X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

# examples are stored in a NumPy matrix, X_train. You have m training examples(3 in the example), and there are n features(4).
# X is a matrix with dimensions (m, n) (m rows, n columns)

# data is stored in numpy array/matrix
print(f'X shape: {X_train.shape}, X type: {type(X_train)})')
print(X_train)
print(f'y shape: {y_train.shape},y type:{type(y_train)})')
print(y_train)

# w is a vector with n elements, each elements contains the parameter associated with one feature
# for demonstration, w and b will be loaded with some initial values thar are near the optimal. w is a 1-D NumPy vector
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

def predict_single_loop(x,w,b):
    """
    single predict using linear regression

    Args: 
        x(ndarray): shape(n,) example with multiple features
        w(ndarray): shape(n,) model parameters
        b(scalar): model parameter
        returns:
        p (scalar): prediction
    """
    n = x.shape[0]
    p =0
    for i in range(n):
        p_i = x[i] * w[i]
        p = p + p_i
    p = p + b
    return p
# get a row from our training data
x_vec = X_train[0,:]
print(f'x_vec shape {x_vec.shape}, x_vec value: {x_vec}')

# make a prediction 
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f'f_wb shape {f_wb.shape}, prediction: {f_wb}')

# even though what we did was rigth, we can be more efficient using vector operation
# as we did in the last lab, we can use np.dot to accelerate

def predict(x,w,b):
    p = np.dot(x,w) +b
    return p
# get a row from our training data
x_vec = X_train[0,:]
print(f'x_vec shape {x_vec.shape}, x_vec value: {x_vec}')

# make a prediction 
f_wb = predict(x_vec, w_init, b_init)
print(f'f_wb shape {f_wb.shape}, prediction: {f_wb}')

def compute_cost(X,y,w,b):
    """
    compute cost
    Args:
        X (ndarray (m,n)): Data, m examples with n features
        y (ndarray (m,)): target values
        w (ndarray (n,)): model parameters
        b (scalar) : model parameter
    returns:
        cost (scalar): cost
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b
        cost = cost + (f_wb_i - y[i])**2
    cost = cost / (2*m)
    return cost
# Compute and display cost using our pre-chosen optimal parameters
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w: {cost}')

def compute_gradient(X, y, w,b):
    """
    Computes the gradient for linear regression
    Args:
        X (ndarray (m,n)): Data, m examples with n features
        y (ndarray (m,)) : target values
        w (ndarray (n,)) : model parameters  
        b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):
        err = (np.dot(X[i], w)+ b) - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw
# compute and display gradient 
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n{tmp_dj_dw}')

def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in) # avoid modifying w within function
    b = b_in

    for i in range(num_iters):
        # calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w, b) ##None

        # Update parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw   ##None
        b = b - alpha * dj_db   ##None

        # save the cost of J at each iteration
        if i < 100000:  # prevent resource exhaustion
            J_history.append(cost_function(X, y, w, b)) 

        # print cost every at intervals 10 times or as many iterations if < 10
        if i % math.ceil(num_iters/10) == 0:
            print(f'Iteration {i:4d}: Cost {J_history[-1]:8.2f} ')

    return w, b, J_history # return final w,b and J history for graphing 
# initialize parameters 
initial_w = np.zeros_like(w_init)
initial_b = 0
# some gradient descent settings
iterations = 1000
alpha = 5.0e-7
# run gradient descent
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b, compute_cost, compute_gradient, alpha, iterations) 
print(f'b,w found by gradient descent: {b_final:0.2f},{w_final}')
m,_ = X_train.shape
for i in range(m):
    print(f'prediction: {n.dot(X_train[i], w_final)+ b_final:0.2f}, target value: {y_train[i]}')

# plot cost versus iteration  
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 4))
ax1.plot(J_hist)
ax2.plot(100 + np.arange(len(J_hist[100:])), J_hist[100:])
ax1.set_title("Cost vs. iteration");  ax2.set_title("Cost vs. iteration (tail)")
ax1.set_ylabel('Cost')             ;  ax2.set_ylabel('Cost') 
ax1.set_xlabel('iteration step')   ;  ax2.set_xlabel('iteration step') 
plt.show()

AttributeError: module 'matplotlib' has no attribute 'get_data_path'